In [4]:
import pandas as pd
from pathlib import Path
import sys
import pickle
sys.path.append('../Data_Collection_Cleaning/')
from data_collection import get_season_lineups
from data_collection import isLatin
from data_collection import get_live_lineups
from unidecode import unidecode

In [ ]:
model = pickle.load(open(filename, 'rb'))

In [5]:
raptor = pd.read_csv(str(Path.cwd()) + '/latest_RAPTOR_by_team.csv')
season_lineups = get_season_lineups('2023')
lineups = pd.concat(season_lineups, axis=0)
lineups["Unnamed: 1"] = lineups['Unnamed: 1'].str.replace('[^\w\s]','', regex=True)
lineups['Unnamed: 1'] = lineups['Unnamed: 1'].apply(unidecode)
lineups.loc[lineups['Rk'] == 'CHO', 'Rk'] = 'CHA'
lineups['Age'] = 2023

todays_lineups = get_live_lineups()
ratings = pd.DataFrame(data={'player_name': raptor['player_name'], 'season': raptor['season'], 'team': raptor['team'], 'rating': (raptor['raptor_total'] + abs(min(raptor['raptor_total']))) * raptor['mp']})
ratings['rating'] = ratings.apply(
    lambda row: row['rating'] / lineups.loc[(lineups['Unnamed: 1'] == row['player_name']) & (lineups['Age'] == row['season']) & (lineups['Rk'] == row['team']), 'G'].item() if len(lineups.loc[(lineups['Unnamed: 1'] == row['player_name']) & (lineups['Age'] == row['season']) & (lineups['Rk'] == row['team']), 'G']) != 0 else None, axis=1)